In [71]:
def compress(data):
    result = bytearray()
    i = 0
    data_len = len(data)
    
    while i < data_len:
        # Look for repeats
        repeat_length = 1
        while i + repeat_length < data_len and data[i + repeat_length] == data[i] and repeat_length < 255:
            repeat_length += 1
        
        # Look for matches
        best_match_length = 0
        best_match_distance = 0
        for j in range(max(0, i - 255), i):
            match_length = 0
            while (i + match_length < data_len and 
                   match_length < 255 and 
                   data[j + match_length] == data[i + match_length]):
                match_length += 1
            if match_length > best_match_length:
                best_match_length = match_length
                best_match_distance = i - j
        
        if repeat_length > 3 or (repeat_length > 1 and repeat_length > best_match_length):
            result.append(repeat_length)
            result.append(data[i])
            i += repeat_length
        elif best_match_length > 2:
            result.append(0)  # Signal for back-reference
            result.append(best_match_length)
            result.append(best_match_distance)
            i += best_match_length
        else:
            result.append(1)
            result.append(data[i])
            i += 1
    
    return bytes(result)

def decompress(data):
    result = bytearray()
    i = 0
    data_len = len(data)
    
    while i < data_len:
        if i + 1 >= data_len:
            raise ValueError(f"Unexpected end of input at position {i}")
        
        count = data[i]
        i += 1
        
        if count == 0:  # Back-reference
            if i + 2 > data_len:
                raise ValueError(f"Unexpected end of input at position {i}")
            length = data[i]
            distance = data[i + 1]
            i += 2
            if distance == 0 or distance > len(result):
                raise ValueError(f"Invalid back-reference: distance {distance} at position {i-2}")
            for j in range(length):
                result.append(result[-distance])
        else:
            if i >= data_len:
                raise ValueError(f"Unexpected end of input at position {i}")
            value = data[i]
            i += 1
            result.extend([value] * count)
    
    return bytes(result)

def compress_hex(hex_string):
    data = bytes.fromhex(hex_string)
    compressed = compress(data)
    return compressed.hex()

def decompress_hex(compressed_hex):
    compressed = bytes.fromhex(compressed_hex)
    decompressed = decompress(compressed)
    return decompressed.hex()

def print_compressed(compressed):
    print(" ".join(f"{b:02x}" for b in compressed))

# Test code
test_strings = [
    map_1_2_top,
    map_1_2_bottom,
    map_3_4_top,
    map_3_4_bottom
]

for original_hex in test_strings:
    # print(f"\nTesting with: {original_hex}")
    original = bytes.fromhex(original_hex)
    compressed = compress(original)
    compressed_hex = compressed.hex()

    # print(f"Original length: {len(original)}")
    # print(f"Compressed length: {len(compressed)}")
    print(f"Compression ratio: {len(compressed) / len(original):.2f}")

    print("Compressed data:")
    print_compressed(compressed)

    try:
        decompressed = decompress(compressed)
        # print("Decompressed data:")
        # print(decompressed.hex())
        print(f"Matches original: {original == decompressed}")
    except ValueError as e:
        print(f"Error during decompression: {e}")
        print("Compressed data (decimal):")
        print(list(compressed))

Compression ratio: 0.89
Compressed data:
02 07 17 2a 01 07 01 07 01 53 02 76 01 51 01 07 01 71 01 07 01 56 01 76 01 56 01 54 02 50 01 51 00 03 0c 01 07 00 03 08 00 03 02 01 55 01 51 02 56 01 76 00 03 07 04 50 01 4c 01 4b 00 08 1b 00 06 15 00 07 12 00 03 21 00 07 31 01 4c 00 0d 60 01 00 01 4b 00 04 3c 00 04 3a 00 03 3e 00 04 41 00 03 1f 08 07 02 00 01 1d 05 07 00 06 06 01 07 01 64 02 60 01 70 01 60 01 60 01 52 01 56 01 53 00 04 07 00 06 0c 01 76 00 04 12 00 05 0e 08 60 01 6b 01 49 01 47 01 6c 00 07 1b 07 60 00 05 12 04 60 00 06 31 01 6b 01 49 00 05 60 01 71 03 00 01 1d 00 03 04 01 2a 01 57 00 05 1d 02 70 05 60 01 70 00 08 1f 05 07 01 00 01 00 01 3c 02 0d 01 2a 01 2d 00 03 91 01 71 01 00 00 03 f7 01 5f 01 04 02 6f 01 7e 03 6f 01 7f 01 6f 01 1b 01 2b 03 04 00 03 0a 00 04 0e 07 6f 01 6d 04 6f 01 04 01 6f 01 7b 01 60 01 49 01 47 01 60 01 6e 01 4e 00 06 10 00 06 04 00 04 12 01 04 00 03 1b 04 6f 00 03 36 01 60 01 59 00 03 7d 00 03 db 01 00 01 00 01 2c 01 3d 04 00 01 07 01 43 01 7e 01 04 01 78

In [25]:
# Example usage
map_1_2_top = """
07072a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a07075376765107710756765654505051765107075450505050505551565676505055505050504c4b505051765107075451565676505050504c4b505051565676517651070754504c072a2a2a2a2a2a2a2a2a2a2a2a004b5156567655505050555050555050504c
072a070707070707070700001d07070707071d070707070707646060706060525653607060606060706060527660706060706060606060606060606060606b49476c60706060527660606060606060606b49476c60606060607060605276606b492a07070707710000001d0000002a576c606060607070606060606070606b49
2a07070707070707070700003c0d0d2a2d072a0771000771075f046f6f7e6f6f6f7f6f1b2b0404046f6f6f7e6f6f6f6f6f6f6f6f6f6f6d6f6f6f6f046f7b604947606e4e6f6f6d6f6f6f6d6f6f6f6d6f60494760046f046f6f6f6f6f04046f605907077107000000002c3d0000000007437e04787f6f6f7e6d7e6f7f7e6f604a
2a07077107070707070700000707070715163d070700070707077e787a7a797a78797a7a1b1c6f6f7f797a7a7a7a7a7a787a7a7a7a7a6d7a7a7a6f040404605947606f7b7a7a6d7a7a7a6d7a7a7a7b6f604947606f7b04047a7a7a7a04046f606907077707000000001d007100000007077f79797a7a784d5e7a7a7e7b6f6149
2a07770d0d2d07070707000007710707252607070700070707077f797a786f6f6f6f7a7a7a7a7a7a7a7a7a6f6f6f7f6f7a787a7a7a7a6d7a7a7a6f6f6d6f606957606f7a7a7a5d6f6f6f5e7a7a7a7a6f604947606f6d7a7a6f6f6f7a7a7a6f605a0d0d3d07000071072a2a0000000007536f787a7879786d787879797f7e6049
2a070707071d0707070700000707072a0d3d2a7107000707077d7e7f6f6f7e04046f6f6f6f7f6f6f7f6f6f7e0404046f0b0c6f7a7a7a5d4e7a7a7a7a6d6f605a68606f7a7a7a6f7a7a7a7a6f4d6e6e6e604947606f6d7a6f047a7a6f7a046f604907710707000000072a2a0000000058617e7a7a7a7a6f6f6f7a0b797a7f604a
2a070707071d07070707000007072c3d0707070707007107077460606060607060604266464666437060606060606070606b6f047a7a6f6f6f6f7a0b5d6e604a68616f4d4e6f6d7a7a7a4d6e5e7a7a04604947606f5d6f7a7a307a046f4d6e60725050515654504c07071d0000710047607f797a7a6f7a6d7a6f1b7a786f6149
2a070771071d07070707000007071d0707070707070007070707664646664440404107070771070744414666664445406361047a7a7a6f7a7a6f7a1b0c6f604947606e5e6d6f5d6e2e2f5e7a6f7a7a04604947606f7a6f04307a306e6f5e6f6b6060606060606b4907071d0000000047606f787a6f7a7a307a7a6f797a6f604a
4b505050517651505050505050505050505050505050505050505050505050505050505050505050505050507656545073606f7a7a7a6f7a7a6f6f6f6f6f605947606f7a5d6f7a7a3e3f7a7a6f7a7a6f604947606f7a6f6e6e3004046f7a0404046f6f6f6f6f6049072a2a0000070748617e797a6f7a3079304e7a79786f6049
476c60606060606060606060606060606060606060606060606060606060606060606060606060606060606060606060606c6f7a7a7a6f6f6f6f7a7a7a6f606a47606f7a7a6f7a7a6d7a7a7a6f4d6e6e604957606f7a7a6f046d7a6f047a6f6f6f046f6f7a6f60492c2a2a0000070747606f787a6f7a7a30795d6f6e6e6e614a
57606f6f6d6f6f7a0b2b046f6f6f6f6f6f6f6f0404046f6f6f6f6f6f6d6f6f1b1c6f6f6f6f6f6f6f6f6f6f6f6f6f6f6f6f6f6f7a7a7a7a7a6f6d7a307a4d606a47606f7a7a7a6f6e5e7a7a6f7a6d7a6f604968606f7a7a046f6d6f7a7a7a6f6c60606b047a6f60493d07077100000047607f787a7a6f7a5d4e7e7a797a6f6059
68606f7b5e7a7a7a0b2b1c047a7a7a04047a7a0b0c7a7a7a7a7a7a4d5e7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a6f5d307a305e605a47606f7a7a4d5e6f6f6f6f7a4d5e7a6f604968616f7a7a7a7a6d7a7a7a7a6f60626360047a6f60725107070707544c47606f797a7a7a6f6f6f787a797a6f6069
68616f7a7a7a7a0b2b1c7a7a7a7a6f04046f0b2b0c6f6f7a7a6f6f6d6f6f6f6f6f6f6f0b0c6f6f7a7a6f6f6f6f6f6f6f6f6f6f7a7a7a7a7a6f7a7a307a7a604947606f6f7a6d7a7a7a7a7a7a6d7a7a6f604947606f6f7a4d6e5e7a7a046f6f604947606f7a6f6b6060520707536b49486079797a7a7a7a7a6d7a7a7a7978605a
68606f7a7a7a7a1b1c7a7a7a7a6f6c6060606060606b6f7a7a6f6c606060606060606060606b6f7a7a6f6c6060606060606b6f6f7a6f6f04046f6e5e7e7e604a47606f6f6f6d6f6f6f6f6f6f6d6f6f6f604947606f6f6f6d6f6f6f6f6f6f6f604947606f7a6f6f04047e6f7f6f605947607878797a78797a6d787a787979614a
58606f7b4e7a7a7a7a7a7a047a6f60624040404063606f7a7a6f6062404040404040404063606f7a7a6f606240404040636b6b6f7a6f6c606060606060606c49476b60606060606060606060606b6f6f6049476b60606060606060606060606c4947606f7a7a7a04047a787b6f606948606f78046f6f6f7e6d7f7e6f6f7f604a
48606f6f6d6f6f6f6f6f6f6f046f60494b50505073606f7a7a6f60725050504c4b50505073606f7a7a6f60725050504c5b63606f7a0b6162404146444045405c5b404040404646444542444063606f6f60495b404646444542444040404040405c47606f7a7a7a7a7a7a7a7a7f605a476b6070606060606b6f6f6c6060606c49
476b6060606060606060606060606c49476c6060606c6f7a7a6f6b6060606b49476c6060606c6f7a7a6f6b6060606b494b73616f7a1b6072505051565450504c00000007070707070707545073606f6f607250505050505050505050505050505073606e6e6e6e4e7a7a04046f60495b4040404040456360046f60624040405c
5b40404046664440404040404040405c47600404046f6f7a7a6f6d6f0404604947600404046f6f7a7a6f046f6f6f6049476c6c6f7a6f6b606060606060606b49077700076452565653537060606c6f6f6b60606060606060606060606060606060606c6f7a7a7a6d7a7a7a046f6072515656765050507360046f60725050504c
4b50505050505050505050517651505073600404046f7a7a7a7a5d6e0404604948606f7a7a7a7a7a7a7a7a047b6f604947606f6f7a6f6f6f6f042b2b04046049001d0007436f6f6f6f6f6f6f6f6f6f6f6f6f1b1c6f6f6f0404046f6f6f6f6f6f307a7a7a7a7a7a6d7a7a7a7a6f6b6060606060606060606c6f6f6b6060606b49
476c6060606060606060606060606060606c6f6f6f6f7a7a7a7a7a6f0b0b604948616f7a7a7a7a7a7a7a7a7a5d6e604957606f7a7a7a7a7a7a7a1b2b2b0c6059071d0007077a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a6f0404046f7a7a7a7a04307a7a7a7a7a7a6d7a7a7a7a6f6f6f6f6f0404046f6f6f6f6f6f6f6f6f6f6149
47606f6f6f6f6f04046f6d6f6f6f6f6f6f6f6f7a7a7a7a7a7a7a7a0b2b2b604947606f7b4e7a7a7a7a7a7a047a6f604968600c7a7a7a7a7a7a7a7a1b2b6f6049073c2d07077a7a7a7a7a7a047a047a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a30047a7a7a7a7a045d4e7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7b6f614a
47606f7b7a7a7a7a7a7a6d7a7a6f6f6f6f6f6f6f6f6f6f6f6f6f6f0b2b2b604957606f6f6d6f6f6f6f6f6f6f046f604968612b0c6f6f6f6f6f6f6f7a1b1c604907711d07536f6f6f6f6f0404046f6f6f6f6f6f6f6f0b0c6f6f6f6f6f6f6f6f306f7a7a6f6f6f04046d6f6f6f6f6f6f6f6f6f6f6f6f0404046f6f6f6f6f6f604a
47606e6e6e6e6f6f6f6f5e7a7a6f6c60606b6060606060606060606060606c49586b6060607060606060606060606c49586b606060606060606b6f7a7a6f604907001d077470606060706060606060606060606060606060606b6f04606263606f7a7a6f6c606060606060606060606060606060606060606042000043606c49
48606f7a7a307a7a7a7a307a7a6f60624040404046664440404040404040405c5b40406644404040404040404040405c5b4040404040404063600c7a7a6f605907001d0707664644404040404040404040404040404040406361046f605947606f7a7a6f6062404040464644454244404040404040404040410707710044405c
48606f7a6f6d7a7a7a7a7a6f7a6f60494b505050505176515050505050505050505050505050504c4b5050505051077107567656545050507360040c7a6f606a07071d07070707073c0d0d0d2d000000000000000000000073606f6f6a004760047a7a6f604900072a2a0000000000000000710000000000000000000000072a
48616f7a6f5d6e6f6f7a4d6e6e6e6049476c60606060606060606060606060606060606060606b49476c6060606052565360706060606060606c1c7a7a6f606907001d0707070707070707071d6452765352765656765360606c046f60655760047a7a6f604900072a2a0000000071000000000000000071000000000000072a
47606f7a6f7a7a6f6f6e5e6f7a6f604947600404046f6f6f6f6f6f6f6f0404046f6f6f6f6f6f604947606f6f04046f6f6d6f6f6f0404046f6f6f6f7a7a0b606907711d0707070707070707071d430404047e6f7f6f7e6f6d7f6f7a04044207436f7a7a6f60490007071d00004b5051565676505051765150505050505050504c
57606f7a6f7a7a6d7a7a7a6f7a6f6072736004047a7a7a7a7a306f6f7a7a7a7a7a7a7a7a6f6f604947606f7a0b0c7a7a6d7a7a7a7a7a7a7a7a7a7a7a7a1b604907001d0707070771070707071d686f7a79787a7a04787a6d797a7a7a04072c0d6f7a7a6f60490071071d0000476c607060606060606070606060606070606b49
68616f7a7a306e5e7a7a307a7a6f6b60606c6f7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a0b6f604947606f7a7a1b0c6f5e7a7a7a7a7a7a7a7a4d6e6e6e6e60490d0d3d0707070707070707071d437f7a7b7a7a0479787a5d6e6e7b7a6f0d3d076f7a046f60490007772a000047600404046f6d6f6f6f6f6f6f6d6f6f6f6f6059
68606f7a4d5e6f6f6f6f7a7a7a6f6f6f6f6f6f7a7a7a7a7a7a7a7a7a7a7a6f6f6f7a7a0b2b6f604947606f7a6f301b306f0c7a7a7a7a7a7a7a6d7a7a7a6f60490707070707070707070707071d677e786d7a0404787a7a797a7a78797e5207536f7a7a6f6049002c2a2a000047600404047a6d7a7a7a7a7a4d5e7a7a6f6f6049
58606f7b6d7a7a7a7a7a7a7a7a6f6f6f6f6f6f7a7a7a7a7a7a306f7a7a6f7a7a7a6f7a0b2b1c604947606e6e6e6e7b0b2b1c7a7a7a7a7a7a7a6d7a7a7a6f60490707070707070707077107073c537f6f6d6f6f6f7f7e6f6f7f7e6f7f7f6a58606f7a7a6f6049001d0707000047606f7a7a7a5d4e7a7a7a7a6d7a7a7a7a6f6049
48606f6f6d6f6f6f6f6f6f6f04046c60606b6f6f6f6f6f6f6f6f6f7a7a6f7a6f7a6f7a1b1c0460494760041b0c6f306f1c7a7a7a7a7a7a7a6f6d6f7a4d6e606907070707070707070707070707744246436646434246466643426646437547606f7a7a6f607250505050505073606e6e6e4e7a5d6f6f6f6f5e7a7a7a7a6f6049
""".replace('\n', '')

map_1_2_bottom = """
74b6060606060606060606060606c62636b606060606060606b6f6a7a7f6a7a7a7f6a7a740b70694740640c1c1a7f6a7a7a7a7a7a7a7a7f603e503e6e5f60696
70707070707070177070707070b4050515701770656765450505050505053706f6a7a7f6b60606060606060606c6f6a7a7d6a7f6a7a7a7a7f6a7a7a7a7f60695
b50404046466440404040404040404c5b5040404646644043606f6a7a7a7f6f6f6a7a7a740f606947406f6a7a7a7a7a7a7a7a7a7a7a7a7a7a7b7a7a7a7b006a5
7017707070707070707070707075c606062565350607060606060606060606c6f6a7a7f6f6f6f6f6f6f6f6f6f6f6f6a7a7d6f6d6a7a7a7d4e4f6a7a7a7f606a6
b405050505050515671505050505050505050505050505053706f6a7a7a7a7a7a7a7a7a7f64006947406f6a7a7a7a7a7a7a7a7a7a7a7d4e6e603f6a7b0b20694
7070707070a27070a2707070707034f6f6f6f6f6f6f6f640f6f640f6f64003c0c0b2c0a7a7a7a7a7a7a7a7a7a7a7a7a7a7d6f6d5e6e2f2e5d5f6e6e6e6e60696
74c60606060606060606060606060606060606060606060606c6f6a7a7a7a7a7a7a7a7a7a7f606947406f6a7a7a7a7a7a7a7a7a7a7a7d6a7a7f6a7b0b2b20694
7070707070705161707070c2d0d0d0f6a7a7a7a7a7a740a7a7a7a740a74003b1b2b1c1a7a7a7a7a7b0c0a7a7a7a7a7a7d4d5f6a7a7e3f3a7a7f6a7a7a7400696
7406f6b1b2b2c1f6d6f6f6f6f6f6f6d6f6f6f6f6f6f6f6f6f6f6f6f6a7a7a7a74040a7a7f6f606947406f6a7a7a7a7a7a7a7a7a7a7a7d6a7a7a7a7b140400695
7070707070705262707070d1701700f6a7a7a7a7a7a7a7a7a740a7a7a7a74003c1a7a7f6f6f6f6b0b2b2c0f6f6f6d4e6e5a7f6a7a7d6a7a7a7f6a7a7a7400694
7406f6b1b2b2c1a7d6a7a7a7a7a7a7d6a7a7a7a7f6f6f6f6f6f6f6f6f6f6f6f64040f6f6f6f606947406f6f6f6f6f6f6404040f6f6f6d6f6f6f6b0b2404006a5
7070707070a27070a2d0d0d3707035f6f6f6f6f6f6f6f6f640f6f64040f6f603f6a787f6c60606060606060606b6e5a7a7a7a7f6e6e5a7a7f6a7a7a7a7f60695
8406f6a7b1c1f6f6d6f6f6f6f6f6d4e5f6f6a7a7a7f6c6060606060706060606060606060606c69475b6b6f6f7c606060606060606060606060606060606c694
7070707070707070707070707085b606060606060606060606060606060606b6f6a797f606260404040404043606f6a7a7a7d4e5f6f6f6f6a7a7d4e6e6e60696
8416f6a7a7f6f6a7d5e4a7a7a7a7d6a7a7f6f6a7a7f60626046644040404040404040404040404c5176416f6f6062614646454040464664404045466044404c5
70700000000000000000000000b5040404646444542444040404040404043606e797a7f706957070d17070707406f6a7a7a7d5e4a7a7a7a7a7a7d6a7a7f606a5
7406f6a7f6f6a7a7a7d6a7a7a7a7d6a7a7a7f6f6a7f60694b4050555050505551570706567701770008606f6e7069400000000001700007017707070d11770a2
a2000000000000000000000000b405050505050505c47000d170707000007416e7a7a7f624707070d17070177506f6a7a7a7a7d6a7a7a7a7a7a7d6a7a7f60696
7506e6e6e6e6e4a7a7d5f6f6f6f6e5a7a7a7a7f6a7f6069474c6060606060606062535060735567077701640e7062715771770707070707070707070d1707070
b405051567156567654505050537c60606060606b694700077a2707070007516f7a7a7e725707070a2a270707606f6f6a7a7a7d6a7a7a7a7a7a7d6a7f6f60696
7406f6a7f6a7d6a7a7f6a7a7a7a7f6a7a7a7a7f6a7f606948406f6f6f6d6f6e7f7f6f640404006a57046c640f6b60606072567350607567070707000a2a27070
74c6060606060607060606060606c6f640f6b7f606940000a2a2707070001786e78797f7a6701770a2a270708506f6f6f6f6f6f6f6f6f6f6f6f6f6f6f6f606a5
7406f6a7f6a7d6a7f6d6a7a7a7a7a7f6a7a7a7f6a7f606947406e7b7a7d6a7a7a7d4e6e6e6e606a67034f6f6f6f6f6f7f6e7f6404040f47070707000a2a27070
7406f6f6f6f6f6f6f6d6f6f6f6f6f6f6a74040f60694000000d1001770707047240000345700707070d1707074b606070606060606060706060606060606c694
7406f6a7f6a7d6a7f6d5e6e2f2a7d4f6e6e6e6e6e6e606948406e6e5a7d6f6f6f6e5b0c0c0f616941785f687a7a7a7a7a7a7a7a797e770707017700070d17070
7506f6a7a7a7a7a7a7d6a7a7a7a7a7a7a7a740f606940000a2a2000000700000000000000000707070d17070b5040404040404040404040404040404040404c5
7406f6a7f6d4d5e6f6a7a7e3f3e6e5f6a7a7a7a740400694750640a7a7f6a7a7a7f6b1b2c1f706a47084408797a7a7a7a79797a7a7a7707070707000c2d37070
8606e6e6e6e4f6f6f6e5b0c0a7d4e6e6e6e6e5f606947000a2a20000007070007070707070177070a2a27070b4051565656715701770656765051567150505c4
7406e6e6e6e5a7a7f6a7a7d6a7a7a7f6a7a7a7f640400694860640a7f6d6a703a7a7f6b1c1f606a670864040f6f6f6f7f6f6e7f6e7f6c77070707000d1707070
8616f6a7a7f6a7a7b0f6b1c0a7d6a7a7b0c0a7f606967070c3d0d2007070707070177070707070c2a2a2707075c606060606062565350607060606060606b694
7506f6a7f6a7a7a7a7f6e6e5a7a7f6a7a7a7a7f6a7f606948616f7a7f6d5039703d4e6e6e6e616967047246466346664340706060606577070707000a2a27070
7606f6a7f6d6c003b1c0f6c1a7d6a7b0b2b2b2f6f470707070a2a2707017707070707070707070d1707070707034f6f7e7f6404040f6e74040f640f6e7f60694
76064040f6a7a7a7d4e5f6f6f6f6a7a7d4e6e6e6e6e606948606f6a7f6a7a70397e5f6a7a7f606967070177070707070706664666464001770707000a2a27070
7606f6a7f6d503a703e6e6e6e6e5b0b2b2b2c1a77070177070a2a2707070707070707070707070d1707070700000e7a7a7a7a7a787a7a7a7a7a7a740b7f60695
86064040f6f6a7a7d5e4a7a7a7a7a7a7d6a7f6f6b0c006948506f6a7a7f6e6e5a7e7a7a7a7f7f470707070707070707070707070700000707070707070d17070
8506f6a7f6a7a703a7a7f6a7a7a7b1c1b1c1a7a77070000000d100000046256535560000001700c3a2a200000000f687a7a7a7a7a7a7a7a7a7a7a7a7d5e60694
76064040a7f6f6a7a7d6a7a7a7a7a7a7d6f6f6b0b2c106948406f6a7d4e5f6f6f687a7a787977070177070701770707070707017700000707070170070d17070
7406f6c0a7f6e6e5a7f6a7a7a7a7a7a7a7a7a7a77070000000c3d2000086e7e7f7a6000000000000a2a200001700f6a7a7a7a7a7a7a7a7a7a7a7a7a7a7a70695
8506f6a7a7a7f6f6f6d6f6f6f6f6f6f6d6f6a7b0c1f606948416c0a7d6a7a7a7a7a7a7a7a7a770707070707070000000000000707070a2707070000070d17070
7506f6b2d4e5f6f6f6a7a7b0b0c0a7f6f6f6f6f6c77070707070d1707076f7b7f6a670007070707070d170700000f6c0a787a7a7a7a7a7a7a7a7a7a7a7a70696
7406f6b7a7a7d4e6e6e5a7b0b0c0a7a7d5e4b0c1b7f606947406b2c0d6a797a7a7a7a7a7a79770707070707070c2a2d0d0d0d2707070d17070a2a27070d17070
7606f6c1d6a7a7a7a7a7b0c1b1b1c0f6a740404006a57070a2a2d3177086f7e7e72570007070a2a270d170707000e7b1c0a7a7b7e4a7a7a7a7a7a740a7f606a5
7406f6f6f6f6d6f6f6f6b0b2b2b2c0f6f6d6b1f6f6f606947506b2c1d6f6e7f6f6f6f7e7f7e7c7707070a2d0d0d370707070c3a2d0d0d3d0d0a2a2d0d0777070
8506f6f6d6f6f6f6f6b0c1f6f6f6f6f64040404006947070a2a2707070472417345770007070a277d0d370707035f7f6e7f7f6f6d6f6e7f6f6f6e7f640f60694
74b60606060606066606060606060606060606060606c69470470606060706060606060706065770707070707070177070707070707017707070707070707070
74b6060606060606060606060606060606060606c69470707070707070707070707070707070d1707070707085b606060606060606060606060606060606c694
b504040404040414174414666664644454244404040404c517703424646466646664146466647070a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2
b50404040404040404646444542444040404040404c5a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2b5040404646444542444040404040404040404c5
""".replace('\n', '')

map_3_4_top = """
4b50515676545550515656765056515654555050505551565156545550504c2a2a2a2a2a2a2a2a2a4b505050515056505654555050505050565056545550504c072a2a2a2a2a2a2a4b5076555150515676545550505050765050555050507650505055555050515056505654555051567654555550765050505076507650504c
476c6060707060606060606060606060606060606060606060606060606b49070707071d07070700476c60606060606060606060606060606060606060606b492a07070707070707686c607060606060707060606060607060606060706060606060607060606060606060606060606070706060606060706060606060606b49
47606f6f6f6f6f6f6f6f6f6f6f6f7a7a7a6f6f6f6f6f6f6f6f0404046f60490d7707071d070777004760040404046d04046e6e6f6f6d6f6f6f6f04046f6f6049070707070707070748606f6f6f6d6f6f6f6f6f6f6f6f6f6f6f6d6f6f6f6f6f6f6f6f6f6f6f6f6f6f6f6f6f6f6f6f6f6d6f6f6f6f6f6f6f6f6f6f6f6f6f6f6049
47606e7b4e7a7a7a7a7a7a7a7a6f7a7a7a7a7a7a7a7a7a7a6f0404046f6049070707071d71073c0d47600404047a6d7a7a7a7a7a4d5e7a7a7a7a04047a6f6049070707070707070747617e7a7a6d7a7a7a7a7a7a4d5e7a7a7a6d7a7a7a6f6f6f6f6f6f6f6f6f6f6f6f6f7a7a7a7a7a6d7a7a7a7a7a7a7a7a7a6d7a7a6f6f6169
47606f7a5d4e6f6f6f6f7a7a7a300b0b0c7a7a7a7a7a7a7a6f6f6f6f6f6069070707071d0707070048616f7a7a7a6d7a7a7a7a7a6d7a7a04047a7a7a7a6f6069070707710707070757607e7a795d4e6f6f6f6f6f6f6f6f6f6f6d7a7a7a6f6c6060706060606060606b6f7a7a7a7a7a6d7a7a7a7a7a7a7a7a7a6d7a7a7a6f6049
48616f7a7a6f7a7a7a7a6f7a7a6f1b2b04040c7a7a7a7a7a7a7a7a7a6f606a070707072a2a07070768606f7a7a7a6d7a6f6f6f6f5e7a7a04047a7a7b7a6f606a070707070707070707436f78797a6f7a7a7a7a7a6d7a7a7a7a6f4e7a7a6f60624040404040404063606f7a7a7a7a7a5d4e6f6f6f6f6f6f6f6f6d7a7a7a6f6049
68606f7a6f6d7a7a7a4d4e6f7a6f0b0404041c7a7a7a7a7a7a7a7a7a6f6169070707072a2a07070748606f7a7a7a5d6f7a7a7a6d6f7a7a7a7a7a7a7a7a6f61690707070771070707070779797a6f7a7a7a7a7a4d5e7a7a7a7a7a6f7a7a6f60494b50505050504c47606f6f6f6f7a7a7a6f7a7a7a7a7a7a7a7a6f4e7a7a6f6149
48606f7a6f6d7a2e2f5e5d6e4e6f1b2b041c7a7a7a7a7a7a7a7a7a797e606a07070771071d07070747616e6e4e7a6f7a7a7a4d5e4d6f7a7a4d6e6e6e6e6e606a0707070707710707070779797e6d7a7a7a7a7a6d7a7a7a7a4d4e7a6f7a6f6049476c6070606b49476b60606b6e6e4e6f6d7a7a4d4e7a7a7a4d4e6f7a7a6f6069
47616f7a6f5d6e3e3f7a7a6f6d6f7a1c7a7a7a7a7a7a7a7a7a797a797e606a07070707071d07070748606f7a6d7a6f6e6e2e2f5d5e6f6e6e5e6f6f6f6f6f606a0707077107070707070778786f5d6e2e2f6e6e2e2f7a7a2e2f5d6e6e6e6e604947606f6f6f60495b404063606f7a6d6f5d6e2e2f6d7a2e2f5e5d6e6e4e6f6049
48606f7a6f7a7a7a6d7a7a6f6d6f7a7a7a7a7a7a7a6f6f6f6f7e797a6f604907070707071d07070768606f7a5d6e6f7a4d3e3f7a7a6f7a7a7a6f6c6060606c490707070707070707075379787f7a7a3e3f7a7a3e3f7a7a3e3f7a7a6f7a6f604947606f7b6f607250504c47606f7a5d6f7a7a3e3f6d7a3e3f7a7a6f7a6d6f6049
68606f7a7a6f6e6e5e7a6f7a6d6f7a797a7a7a7a7a7e6c6060420707436c4907072a2a0d3d07070768606f7a7a7a6f7a6d7a6d7a7a6f7a7a7a6f60624040405c070771070707070758607e786f7a7a7a6d7a7a6d5d6e6e5e7a7a7a6f7a6f606947606f6d6f6b60706b4947606f7a7a6f7a7a6d7a5d6e5e7a7a7a6f7a6d6f6149
68606f7a4d5e6f6f6f6f7a4d5e307a7a797a797a0b0b60624107070707445c07072a2a070707070758606f7a7a7a4d6f5e7a5d4e6f7a7a7a4d6e60590707072a070707070707070768617e7a7a7f7a7a6d7a7a6d7a7a7a7a7a7a6f7a7a6f606a47606f5d4e6f6f6f604947606f7a7a7a6f6e5e7a7a7a7a7a7a6f7a7a6d6f6069
58606f7b5e7a7a7a7a7a7a6d7a6f797879797a0b2b2b60490707070707070707071d07070707074b73606f7a7a4d5e7a6f6f6f6f7a4d6e6e5e6f60690777712a070707070707070748606f7a7a7a6f6e5e7a7a6d7a7a7a7a7a6f4d6e6e6e616947607a0b6d7a7a6f606947606f7a7a4d5e6f6f6f6f6f6f6f6f7a4d6e5e6f6049
47606f6d6f6f6f6f6f6f6f6d6f6f7a787a797e1b2b2b60490707077107070707071d0707070707476c6c6f6f7a6d7a7a7a7a7a7a7a6d7a7a6f6f606a071d072a070707077107070748607e7a4d6e5e6f6f6f6f6f6f6f6f6f6f7a6d7a7a6f606a47600b2b5d4e7a6f606a47606f7a7a6d7a7a7a7a7a7a7a7a7a7a6d7a7a6f6069
476b60606060606060606060607042070707437060606c490707070707070707073c2d0771070747606f6f6f6f6d6f6f6f6f6f6f6f6d6f6f7b6e6069071d072a070707070707070748616f7a6d7a7a7a7a7a7a6d7a7a7a7a7a4d5e7a7a6f606a47601b1c7a6d7a6f616947606e6e6e5e7a7a7a7a7a7a7a7a7a7a6d7a6f6f6149
5b404040454166464646444045410707070707444066405c070707070707070707071d0707070768616f7a7a7a30303030303030303030303030605a073c0d2a070707070771070768606f6f6d6f6f6f6f6f6f6f6f6f6f6f6f6d6f6f6f6f604947606f7a04047a0b606a47606f6f6f6f6f6f6f6f6f6f6f6f6f6f6d6f6f6f6049
2a0707070707071d0707070707070707070707070707072a070707070707070707072a2a07070748606f7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a6f616a0707072a2a2a2a0707070707476b60607060606060606070606060706060606060606c4947606f7a04040b2b606a476b6060606070606060606060706060606060606c49
2a0707070707071d0707070707070707070707070707072a070707070707070707072a2a0d0d0d48606f7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a6f605a7107070707072a07070707075b404040404040404166406641664066444040464040405c47606f7a04041b2b60495b40404166406644404041664066444040464040405c
2a07072a2a0d0d3d0707070707070707077107070707072a07070707070707070707070707070747606f6f6f6f6f6f6f6f6f6f6f6f6f6f6f7a6f604a0707070707072a0707070707076476766507074b5050515056505654555050505050505073606f7a04047a6f60725050515676545550565056545550545550505050504c
2a07072a2a710707070707710707070707070707070707072a0707070707070707070707070707476b606060607060606060606060606b6f7a6f606a0707070707072a070707070707436f7b6a0d0d476c606060606060606060606070606060606c6f7a7a6d7a6f6b6060606060707060606060606060606060606060606b49
2a0707071d070707070707070707070771070707070707072a07070707710707070707070707075b404041664066444040464040406360047a6f60690707070707072a710707070707077e7e6a070757616f7e6f6f6f0b2b2b0c6f6f6f6d6f6f6f6f6f7a0b6d7a6f6f6f6d6f6f6f6f6f6f6f6f6f6f6f6f6d6f6f6f6f6f6f6069
2a0707071d070707070707070707070707070707070707072a0707070707070707070771070707074b5055505050555656545050507360047a6f605a0707070707072a070707710707076f6f6a070707437f6f7a7a7a1b2b2b1c7a7a4d5e7a7a7a7a7a7a1b6d7a7a7a7a5d4e7a7a7a7a7a7a7a7a7a7a7a6d7a7a7a7a6f6f6049
2a0707711d070707070707070707070707070707070707072a070707070707070707070707070707576c6060606060606060606060606c6f7a6f614a0707070707072a070707070707536f6f6a0707070779787a040404041c7a7a04047a047a4d6e040404047a7a7a040404047a7a7a04047a047a7a7a047a7a7a7a7a6f6049
2a0707071d070707077107074b50555050505156765455504c0707072a070707070707070707070707437e6f7f7f6f7f6f6f6f6f6f04046f7a6f604a0707072a2a2a2a070707070758607f7f6a0707070779797a040404047a7a7a040404046e5e7a040404047a7a7a040404047a7a7a040404047a7a7a7a7a7a7a7a7a6f6169
2a0d0d0d3d07070707070707486c6060606060607070606b4907072a07070707070707070707070707077e7a7a7a7a7a7a7a7a7a7a7a047a7a6f60490771072a070707070707070748617e6f6a070707077978787a6d7a7a7a7a7a7a7a7a7a7a7a7a7a7a6d7a7a7a7a0b2b2b0c7a7a7a7a6d7a7a7a7a7a7a7a7a7a7a7a6f6049
2a070707070707070707070757606f7e7e6f7f6f040404604907070707070707070707070707070707077e7a7a7a7a7a0404047a7a7a7a7a7a6f604a0707072a070707070707075473607f6f6a070707537e79797a5d4e7a7a7a7a7a7a7a7a7a7a7a7a7a6d7a7a7a7a1b2b2b1c7a7a7a4d5e7a7a7a7a7a7a7a7a7a7a7a6f6049
2a070707070707070707710707437e7a7a7a7a6f040404604907070707070707070707710707070707536f7e6f7f6f6f040404046f6f6f6f6f6f61490707072a0771070707646070606c6f6f6a070758606f7978040404047a4d6e040404047a7a7a7a0404047a7a7a040404047a4d6e040404047a7a6f040404047a046f6049
2a0707070707070707070707000078787a7a7a6f6f6f6f604907070707070707072c0d0d0d2d0707586b606060606060606060606060706060606c490707072a0707070707686f6f7f7e6f7e6a2d0748617e7a790404046e6e5e7a04047b047a7a7a040404047a7a7a047a04046e5e7a040404047a0b0b047b04047a04046049
2a07070707070707070707070707787a7a7a307a7a7a6f60690d0d2d07070707071d0707073c0d0d5b4045406643454045464666444040404040405c0707072a0707070707677f7e7e7e7f6f6a3c0d48607e7f7a7a6d7a7a7a7a7a7a7a0b0c0c0c7a7a7a6d6f7a7a7a7a7a7a7a7a7a7a7a7a7a7a0b2b2b0c7a7a7a7a7a6d6069
2a07070707070771710707070707797a7a307a307a7a6f606a07071d07070707071d07070707070707070707070707070707070707070707071d07070707072a0707070707686f6f6241466675070068616f6f7e4d5e6f6f6f6f6f6f6f1b2b2b1c6f6f6f5d4e6f6f6f6f6f6f6f6f6f6f6f6f6f6f1b2b2b1c6f6f7a7a4d5e6049
2a077107070707070707070707536f797a7a307a7a7a6f616907073c0d0d2a07072a07070707070707070707070707070707070707070707071d07070707072a072c0d0d0d686f7f7251765365070748606060606060606060606060607060606060606060606060606060606060706060606060606060606b04046e5e6f6069
2a070707070707070707070758607e7a7a7a7a7a7a7a6f606a07070707070715160707070707070771070707070707070707070707710707071d07070707072a0d3d070707686f7e7f7f6f7e6a07075b404040404040404040404166406644404046404040404040404040416640664440404640404040636004047a7a6f6169


""".replace('\n', '')

map_3_4_bottom = """
a27070707070b40555550505370603030303a7a7a7a7f606a67070707070705262707070707070707070707070701770707070707070707070c3d0d0777070a2
707070177086f6e7e7e7e7f7a67070b40515655565450515056505654555050515656505156515656745550505156505654505556545c47406f640a7a7f60694
a2a2a270707074c60607070606c6f6a7404003a7a74040069470707070c2a27070a2d270707070707070707070707070707070707070707070177070707070a2
707070707047060706b6f6f69670c284c6060607060606060606060606060607060606060606060607070607060606060606060606b6957416f6404040f60694
a2a2a27070707406f6404040f6f6f6a7a7a74003030303069470707070d100707070d170707070707070707070707070707070707070707070707070707070a2
70707070700000443606f6e7a6d0d37416f6f6e7e7f6f6404003b2b2b2c1f6f6f6f6f6f6f640404003f6f6404040f6f6d6c1c1f6f606a6740640a703a7f60694
a270d17017707406f6a7a7a7a7a7a7a7a7a7a7a797a7f6069470707070d100707070d170b40505055515707070704555050505551565674555056545550505c4
70707070700000708416f7f79670707506e78797a7a7a7a7404003b1c1a7a740a7a7a7a7a7a740b10303a7a7a7a7b0a7d603a7a7f616967406f603a703e60696
a270d17070708416f6a740a7a7a7a7a7a7a7a797a797f7069470707070d100707070d17074c6060607062570703507060606070606060707060606060606b694
70701770707070707516f6f79670707034878787a7a7a7a7a7a7a703b0a7a7a7a7a703c0a7a7a7a7b103a7a7b040b2c003a703a7f606947406f6b703a7400694
a270d17070708416f6a7a7a7a7a7f6f6f6f7f6f7a797f7069470707070c3d2707070c3d07406f6f6f6f7f69787f7f6f6f6f6f6f6f6f6f6f6404040f6f6f60694
70707070707070000086f6e7a67070707087a78797a7a7a7a7a7b0c003a7a7a7a7a7a703c0a7a7a740c003a7b040b2c1c103a7a7f606a67406f6f64040400694
a2d0d37070708606c0a7a7a7a740f6c606060624707034c6947070707070d170707070708416f603a7a7979787a7a7a7b0a7a7a7a7a7a7a7404040a703f61694
7070707070a270000086e7e7a6707070708797a787a7a7a7a7a7b0b203a7a7a7a7a7a703b0a7a7a7a7b103a7b1b2c1b0b0c0a7a7f6069674b60606060606c694
a2707070a2708406b2c0a7a7a7a7f6062604147070707044c57070707070d170707070708606f6a7a7a7a797a7a7a7b0b2c0a7a7a7a7a7a7a7a7a7a7a7f60694
70707070a270a2707076e7e7a670707070878787a7a7a7a7a7a7b0c103a7a7a7a7a7a70340a7a7a7a7a70303a7a7a7b1404040c0f60694b504146444545404c5
a2707070a2a27416b2c1a7a7a7a7f6069470707070707070707070177070c3d0d0d0d0d08406f6a740a7a7a7a7a7a7b1b2b2c0a7a7a7a7a7a7a7a7a7a7f60627
0505c4c2d0a270707086f7f6a670707035f68797a7a74040a7a7b103c0a7a7a7a7a740034040c0a7a7a7b1c003a7a7b14040b240f606960070707070707070a2
a270177000a28406b2c0a7a7a7a7f60694701770707070707070707070707070177070707416f6a7a7a7a7a7a7a7a7a7b1b2b2c0a7a7a7a7a7a7a7a7a7b0b606
06b694d1707070707047e7f65770708506f7a7a7a7a74040a7a703b0b2c0a7a7a7a74003b0c1a74040a7a7b10303a7a7b1b1b140f606a60070177070707070a2
a270707000708606b2c1a7a7a7a7f606270515656745550505c4707070707070707070708406f6a7a7a7a7a7a7a7a7a7b1b2c1c1a7a7a7a7a7a7a7a7a7b1b240
b206a4a2a2a2a2a2a2000000007070841640a7a7a7a7a7a7a703b0c1b1c0a7a7a7a74003c1a7a7a740a7a7a7a7a7a7a7a7a7a7a7f616960070707077707070a2
a270707070708606c1a7a7a7a7a7f6b60606060607070606b694707070707070707070708606f6a7a7a7a7a7a7a7a7a7a7b1c1a7a7a7a7a7a7a7a7a7a7b0c0b2
b2069670707070000000000000707086064040a7a7a7a7a7a703b1b7b1b2a7a7a7a70303a7a7a7a7a7a7a7a7a74040a7a7a7a7a7f606a60070a2707070a270a2
a270707070708506f6a7a7a7a7a7f6f6f6f6f6f6404040f60694707070707070707070708616f6a7a7a740a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7a7b1b2
b206947070707000007070707070708406404040a7a7a7a7a7a703b1b2c0a7a7404003a7a7404040a7a7a7a7a7a7a7a7a7a7a7f6f606a60070701770707070a2
a270707070707406f6a7a7a7a7a7a7a7a7a7a7a7404040f60694d0d0d0d0d270707070708506f603a7a7a7a7a70340404003a7a7a7a7a7a7a7a7a740a7a7a7b1
c116967070701700707070707070707416404040a7a7a7a7a7a7a703b1c1a7a7404003a7a74040f6f6f6f6f6f6f6f6f6f6f6f6f6f606940070707070707070a2
a270701770707406f6a740a7a7a7a7a7a7a7a7a7a7a7a7f6069470701770d170707070707406f6f6f6f6f6f6f60340404003f6f6f6f6f6a7a7a7a7a7a7a74040
f606a67070707000007070707070708406f6a7a7a7a7a7a7a7a7a7a7a7a7a7a7a703a7a7a7a7a7f6c6060607060606060606070606c6940070707070177070a2
a270707070708616f640a7a7a7a7a7a7a7a7a7a7a7a7a7f60696707070a2a2700070177074b6060606060607060606060707060606b6f6a7a7a7a7a7a7f64040
f616967070707070707070707070708606f6a7f6e7f6f6f6f6f6f6f6f6f6f6f6f603f6f6a7b7a7f60626146466646644041464646604c50070707070707070a2
a2a27070707084064040a7a7a7a7a7a7a7a7a7a7a7a7a7f606a670707000a2a200707070b55404541466046644040464045404543606f7a740a7a7a7a7f6c606
06c6947070707070707070707070708606e7b7f6c606060606070606060606060606b6f6a7a7a7f60696707070707070707070701770707070707070707070a2
a2a2707070708406f6f6f6f6f6f6f6f6f6f6a7a703a7a7f61696707070707070707000a200707070707070d170707070707070707516f6a7a7a7a7a7a7f60626
0404c57070700000007070707070708506f6e7f6062614660466440404646644043606f6f6f6f6f60694707770707070707070707070707070a27070a27070a2
a2a27070707074b60606060606060606b6f6a703a703a7f606a670707070707070707070a2707070707070d170177070707070707034f787a7a7a7a7a7f60695
70707070701700000070000000000074b6060706c694707070707070707070707074b60606070606c694707070a27017707070707070707070705161707070a2
a27070177070b504545454146604043606f6a7a703a7a7b006a67070707017707070707070a27070a2d0d0d3707070707070707070709787a740a7a7a7f616a5
7070a2707070000000707070707070b56604666644c57070701770707017707070b504041464666466c57070a2707070707070707070707070705262707070a2
a27070707070707070707070d17070740640a7a7a7a7b0b20694707070707070707070c2d0a2d0a270a2707070707070707070707070a797a7a7a7a7a7f616a4
000070a27070707070707070707070707070707070707070707070707070707070707070707070707070707017707070701770707017707070a27070a27070a2
a27070707070701770c277d0777070740640f6f6f6f6b2b206947070707070707070707770a27070a2d2707070707070701770707035f7f6e7f6f6f6f6f606a4
70777070707070707070177070707070707070707070000000a2a2a2a2a2707070707070707070707017707070707070707070707070707070707070707070a2
a27070707070707070d1707070707074b606060606060606c6947070707070707070707070a2707070d17070707070707070707085b60606060706060606c694
70707070707070707070707070707070707070707070000000a2707070a27070701770707070707070707070707070707070707070707070707070707070a270
70a2a2a2a2a2a2a2a2a2a2a2a2a2a2b5040454146604660404c5a2a2a2a2a2a2a2a2a2a2a270a2a2a2a2a2a2a2a2a2a2a2a2a2a2b504146604664404046404c5
a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2707070a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a2a27070
""".replace('\n', '')

logo = """
eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee3eeeeeeeeee
33333333e33333333e33333333e33333333e33333333e33333333e33333333ee33eeeeee33333333e33333333e33333333e33333333e33333333e33eeee333ee
33333333e33333333e33333333e33333333e33333333e33333333e33333333ee33eeeeee33333333e33333333e33333333e33333333e33333333e333ee333eee
eeeeee33eeeeeee33e33eeee33eeee33eeee33eeee33e33eeeeeee33eeee33ee33eeeeee33eeeeeee33eeee33eeeeeee33eeee33eeeeeeeeeeeeee333333eeee
33333333e33333333e33eeee33eeee33eeee33eeee33e33eeeeeee33eeee33ee33eeeeee33eeeeeee33eeee33e33333333eeee33eeee33333333eee3333eeeee
33333333e33333333e33eeee33eeee33eeee33eeee33e33eeeeeee33eeee33ee33eeeeee33eeeeeee33eeee33e33333333eeee33eeee33333333eee3333eeeee
33eeeeeee33e333eee33eeee33eeee33eeee33eeee33e33eeeeeee33eeee33ee33eeeeee33eeeeeee33eeee33e33e333eeeeee33eeeeeeeeeeeeee333333eeee
33eeeeeee33ee333ee33333333eeee33eeee33333333e33333333e33333333ee3333333333333333e33333333e33ee333eeeee33eeee33333333e333ee333eee
33eeeeeee33eee333e33333333eeee33eeee33333333e33333333e33333333ee3333333333333333e33333333e33eee333eeee33eeee33333333333eeee33eee
33eeeeeeeeeeeee33eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee33eeeeeeeeeeeeeeeeeeeeeeeeee3eee
3eeeeeeeeeeeeeee3eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee3eeeeeeeeeeeeeeeeeeeeeeeeeeeeee
""".replace('\n', '')

In [41]:
compressed_hex

b'\xee\xb9\x01\xe3\x85;3\x83\x01\xe3\x83\x05>\x96\t\xee3\x9a$>\xee\xe3\x83 \xba@\x83]>\x85\x83\x84\x8d>\x83i\x8a\x03\x83\\\x87\x0f\x85\x80\x85!\x84\x03\x88$\x88\xbb\x8c\xc0\xa3@\x85\xd2\x85g\x84\xdb\xe3\x84\x7f\xc1@\x86\xbd>\x8b\xbd\x9b\x80\x83\xf4\x90\xc0\x85\xfd\x83\x18\x8d\x9c\x86\xd2\x84\xe1\x84\xe0\x8b\xe4\x87s\x86\xf6\x84\x0e\x87\xdc\x87.\xa1@\x84\x81\x87\xc0\x83\xfe\x849\x86\x8d\x85\xf1\xa4\x03\x88\xbd\x86\xc4>\x83\x02\x88\n\xa6A\xe3\x8fh'

In [28]:
print_compressed(compressed_hex)

eeb901e3853b338301e383053e9609ee339a243eeee38320ba40835d3e8583848d3e83698a03835c870f858085218403882488bb8cc0a34085d2856784dbe3847fc14086bd3e8bbd9b8083f490c085fd83188d9c86d284e184e08be4877386f6840e87dc872ea140848187c083fe8439868d85f1a40388bd86c43e8302880aa6
41e38f68


In [26]:
len(compressed_hex)/2

132.0

In [37]:
compressed

NameError: name 'compressed' is not defined